# Attention Cues
:label:`sec_attention-cues`

本書にご注目いただき、ありがとうございます。注意（アテンション）を向けるとは希少な資源である。今現在、あなたはこの本を読み、それ以外を無視している。したがって、お金と同様に、あなたが注意を向けることは機会費用で支払われている。今、あなたの注意を向けるという投資が価値あるものであるように、私たちは注意を丁寧に払うという高いモチベーションで、素敵な本を作り上げた。アテンションは人生のアーチにおける要であり、あらゆる作品の例外性の鍵を握っている。

経済学は希少資源の配分を研究するので、人間の注意を限られた貴重な交換可能な商品として扱う注意の経済の時代である。それを利用するために、数多くのビジネスモデルが開発されてきた。音楽や動画のストリーミングサービスでは、その広告に注意を払うか、お金を払って広告を隠すかのどちらかである。オンラインゲームの世界で成長するためには、新しいゲーマーを惹きつけるバトルに参加するために注意を払うか、瞬時に強くなるためにお金を払うか、どちらかである。タダで手に入るものはないのある。

つまり、私たちを取り巻く環境には、情報が不足しているのではなく、注意が不足しているのです。視神経は1秒間に108ビットの情報を受け取っており、脳が処理できる量をはるかに超えている。幸いなことに、私たちの祖先は経験（データ）から、すべての感覚入力が同じではないことを学んできた。人類の歴史を通じて、興味のある情報の一部だけに注意を向けるという機能によって、私たちの脳は、捕食者や獲物、仲間の発見など、生存や成長、社会生活のために、より賢く資源を配分することができるようになった。


## Attention Cues in Biology

To explain how our attention is deployed in the visual world,
a two-component framework has emerged
and been pervasive.
This idea dates back to William James in the 1890s,
who is considered the "father of American psychology" :cite:`James.2007`.
In this framework,
subjects selectively direct the spotlight of attention
using both the *nonvolitional cue* and *volitional cue*.

The nonvolitional cue is based on
the saliency and conspicuity of objects in the environment.
Imagine there are five objects in front of you:
a newspaper, a research paper, a cup of coffee, a notebook, and a book such as in :numref:`fig_eye-coffee`.
While all the paper products are printed in black and white,
the coffee cup is red.
In other words,
this coffee is intrinsically salient and conspicuous in
this visual environment,
automatically and involuntarily drawing attention.
So you bring the fovea (the center of the macula where visual acuity is highest) onto the coffee as shown in :numref:`fig_eye-coffee`.

![Using the nonvolitional cue based on saliency (red cup, non-paper), attention is involuntarily directed to the coffee.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/eye-coffee.png)
:width:`400px`
:label:`fig_eye-coffee`

After drinking coffee,
you become caffeinated and
want to read a book.
So you turn your head, refocus your eyes,
and look at the book as depicted in :numref:`fig_eye-book`.
Different from
the case in :numref:`fig_eye-coffee`
where the coffee biases you towards
selecting based on saliency,
in this task-dependent case you select the book under
cognitive and volitional control.
Using the volitional cue based on variable selection criteria,
this form of attention is more deliberate.
It is also more powerful with the subject's voluntary effort.

![Using the volitional cue (want to read a book) that is task-dependent, attention is directed to the book under volitional control.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/eye-book.svg)
:width:`400px`
:label:`fig_eye-book`


## Queries, Keys, and Values

注意の展開を説明する非自発的注意と自発的注意の手がかりに触発され、以下ではこの二つの注意の手がかりを取り入れた注意機構設計の枠組みを説明する。

まず始めに、非ボリショナルキューのみが利用可能なより単純な場合を考える。感覚入力に対する選択を偏らせるためには、単にパラメータ化された完全連結層や、パラメータ化されていない最大または平均プーリングを用いることができる。

したがって、注意機構をそれらの完全連結層やプーリング層と区別するのは、自発的な手がかりを含むことである。注意機構の文脈では、自発的な手がかりをクエリーと呼ぶ。このようなクエリが与えられると、注意機構は注意プーリングによって感覚入力（例えば中間的な特徴表現）に対して偏った選択を行う。これらの感覚入力は注意機構の文脈では値と呼ばれる。より一般的には、全ての値はキーと対になっており、これはその感覚入力の非揮発的な手がかりと考えることができる。numref:fig_qkvに示すように、与えられたクエリー（自発的手がかり）がキー（非自発的手がかり）と相互作用できるように注意プーリングを設計することができ、これにより値（感覚入力）に対するバイアス選択が導かれる。

![Attention mechanisms bias selection over values (sensory inputs) via attention pooling, which incorporates queries (volitional cues) and keys (nonvolitional cues).](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/qkv.svg)
:label:`fig_qkv`

Note that there are many alternatives for the design of attention mechanisms.
For instance,
we can design a non-differentiable attention model
that can be trained using reinforcement learning methods :cite:`Mnih.Heess.Graves.ea.2014`.
Given the dominance of the framework in :numref:`fig_qkv`,
models under this framework
will be the center of our attention in this chapter.


## Visualization of Attention

Average pooling
can be treated as a weighted average of inputs,
where weights are uniform.
In practice,
attention pooling aggregates values using weighted average, where weights are computed between the given query and different keys.


In [1]:
%use @file[../djl.json]
%use lets-plot

In [2]:
val manager = NDManager.newBaseManager()

To visualize attention weights,
we define the `showHeatmaps` function.
Its input `matrices` has the shape (number of rows for display, number of columns for display, number of queries, number of keys).


For demonstration,
we consider a simple case where
the attention weight is one only when the query and the key are the same; otherwise it is zero.


In [19]:
val attentionWeights = manager.eye(10).reshape(Shape(1, 1, 10, 10))
val matrix = attentionWeights.get(0,0)
val seriesX = mutableListOf<Long>()
val seriesY = mutableListOf<Long>()
val seriesW = mutableListOf<Float>()
for(i in 0 until matrix.shape[0]) {
    val row = matrix.get(i)
    for(j in 0 until row.shape[0]) {
        seriesX.add(j+1)
        seriesY.add(i+1)
        seriesW.add(row.get(j).getFloat())
    }
}
val data = mapOf( "x" to seriesX, "y" to seriesY)
var plot = letsPlot(data)
plot += geomBin2D(drop=false, bins = kotlin.Pair(10,10), position = positionIdentity){x="x"; y = "y"; weight = seriesW}
plot + ggsize(500, 500)

In the subsequent sections,
we will often invoke this function to visualize attention weights.

## Summary

* Human attention is a limited, valuable, and scarce resource.
* Subjects selectively direct attention using both the nonvolitional and volitional cues. The former is based on saliency and the latter is task-dependent.
* Attention mechanisms are different from fully-connected layers or pooling layers due to inclusion of the volitional cues.
* Attention mechanisms bias selection over values (sensory inputs) via attention pooling, which incorporates queries (volitional cues) and keys (nonvolitional cues). Keys and values are paired.
* We can visualize attention weights between queries and keys.

## Exercises

1. What can be the volitional cue when decoding a sequence token by token in machine translation? What are the nonvolitional cues and the sensory inputs?
1. Randomly generate a $10 \times 10$ matrix and use the softmax operation to ensure each row is a valid probability distribution. Visualize the output attention weights.
